In [1]:
import datalabframework as dlf

In [2]:
dlf.project.rootpath()

'/home/natbusa/Projects/dsp-titanic/src'

#### Init Spark

In [3]:
engine = dlf.engines.get('spark')
spark = engine.context()

In [4]:
#print out name and version
'{}:{}'.format(engine.info['context'], spark.sparkSession.version)

'spark:2.3.1'

In [5]:
df = engine.read('.etl.features.train')

In [6]:
df.show(truncate=False)

+---+-----+----------------------------------------------------------------+
|id |label|features                                                        |
+---+-----+----------------------------------------------------------------+
|1  |0    |(19,[0,1,2,5,10,16,17],[22.0,7.25,1.0,1.0,1.0,1.0,1.0])         |
|2  |1    |(19,[0,1,3,5,10,18],[38.0,71.2833,1.0,1.0,1.0,1.0])             |
|3  |1    |(19,[0,1,2,4,10,17],[26.0,7.925,1.0,1.0,1.0,1.0])               |
|4  |1    |(19,[0,1,3,5,10,17],[35.0,53.1,1.0,1.0,1.0,1.0])                |
|5  |0    |(19,[0,1,2,4,10,16,17],[35.0,8.05,1.0,1.0,1.0,1.0,1.0])         |
|6  |0    |(19,[0,1,2,4,10,16],[28.963899873494455,8.4583,1.0,1.0,1.0,1.0])|
|7  |0    |(19,[0,1,3,4,10,16,17],[54.0,51.8625,1.0,1.0,1.0,1.0,1.0])      |
|8  |0    |(19,[0,1,2,8,11,16,17],[2.0,21.075,1.0,1.0,1.0,1.0,1.0])        |
|9  |1    |(19,[0,1,2,4,12,17],[27.0,11.1333,1.0,1.0,1.0,1.0])             |
|10 |1    |(19,[0,1,5,10,18],[14.0,30.0708,1.0,1.0,1.0])                   |

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
pipeline = Pipeline(stages=[lr])

model = pipeline.fit(df)
prediction = model.transform(df)

In [8]:
prediction.select('label', 'prediction','probability').show(truncate=False)

+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|0    |0.0       |[0.8851298504855403,0.11487014951445967] |
|1    |1.0       |[0.0689883852034645,0.9310116147965354]  |
|1    |1.0       |[0.39961202844990046,0.6003879715500996] |
|1    |1.0       |[0.08200469956317774,0.9179953004368222] |
|0    |0.0       |[0.9318137062197771,0.06818629378022292] |
|0    |0.0       |[0.8826418619093687,0.11735813809063134] |
|0    |0.0       |[0.7436267474390812,0.25637325256091886] |
|0    |0.0       |[0.9632642722416761,0.036735727758323775]|
|1    |1.0       |[0.41128199780062663,0.5887180021993734] |
|1    |1.0       |[0.08626684320344105,0.913733156796559]  |
|1    |1.0       |[0.15456883133788948,0.8454311686621105] |
|1    |1.0       |[0.20025482614849038,0.7997451738515096] |
|0    |0.0       |[0.8826901205527755,0.11730987944722451] |
|0    |0.0       |[0.979

In [9]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8636356373629883


In [10]:
model.write().overwrite().save(path='lr2')

In [11]:
from pyspark.ml import PipelineModel
m2 = PipelineModel.load(path='lr2')

In [12]:
prediction = m2.transform(df)
metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8636356373629883
